# 제2장 데이터 요약과 시각화 (3)
### __기술통계분석__


---
### 사전 작업





* 실습에 사용할 데이터를 GitHub 저장소로부터 가져오기 위하여 다음과 같은 명령을 코랩 코드 셀에서 실행

In [ ]:
!git clone https://github.com/BizStat/BizDataAnalysis.git

* 한글 폰트 설치와 matplotlib 환경 설정

In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -f -v
!rm ~/.cache/matplotlib -rf

* 한글 그래프 환경 적용을 위해 세션 다시 시작
  - 위의 메뉴에서 [런타임] -> [세션 다시 시작] 을 선택하거나 [Control] + M 을 누르면 됨

* 설치된 한글 폰트 목록 확인

In [ ]:
!fc-list :lang=ko

* matplotlib에 사용할 한글 폰트 설정
  - 앞의 한글 폰트 목록으로부터 자신이 사용하고자 하는 폰트이름을 복사하여 지정

In [ ]:
from matplotlib import rc
rc('font', family='NanumMyeongjo')

* 자주 사용하는 라이브러리 가져오기

In [ ]:
import pandas as pd

---

### 분석 사례 : 강우량

In [ ]:
df3 = pd.read_csv('/content/BizDataAnalysis/DATA/서울지역_일일_최대강수량.csv')

In [ ]:
df3.head()

In [ ]:
# @title 일 최다 강수량(mm)

from matplotlib import pyplot as plt
df3['일 최다 강수량(mm)'].plot(kind='hist', bins=20, title='일 최다 강수량(mm)')
plt.gca().spines[['top', 'right',]].set_visible(False)

In [ ]:
# @title 일시 vs 일 최다 강수량(mm)

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  xs = series['일시']
  ys = series['일 최다 강수량(mm)']

  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = df3.sort_values('일시', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('일시')
_ = plt.ylabel('일 최다 강수량(mm)')

In [ ]:
df3.일시.min()

In [ ]:
df3['일시'].max()

In [ ]:
df3[df3['일 최다 강수량(mm)'] > 300].일시

In [ ]:
df3[df3['일 최다 강수량(mm)'] > 350].일시

### 서울시의 대기오염 데이터 분석

* 데이터 가져오기

In [ ]:
seoul_air = pd.read_csv('/content/BizDataAnalysis/DATA/AIR_HOUR_2022.csv')
seoul_air.head()

* 숫자로 저장된 측정일시를 파이썬의 datetime 객체로 변환

In [ ]:
seoul_air['측정일시']

In [ ]:
seoul_air['측정일시'] = seoul_air['측정일시'].astype('str')

In [ ]:
seoul_air.head()

In [ ]:
import datetime as dt

* apply 함수를 사용하지 않고 반복문을 사용하여 변환

In [ ]:
for i in seoul_air.index:
  seoul_air['측정일시'][i] = dt.datetime.strptime(seoul_air['측정일시'][i],'%Y%m%d%H')

In [ ]:
seoul_air.head()

* apply 함수를 사용

In [ ]:
seoul_air['측정일시'] = seoul_air['측정일시'].apply(lambda _ : dt.datetime.strptime(_,'%Y%m%d%H'))

In [ ]:
seoul_air.head()

* 측정소 정보와 측정항목 정보를 가져오기

In [ ]:
측정소정보 = pd.read_csv('/content/BizDataAnalysis/DATA/서울시 대기오염 측정소 정보.csv',encoding='EUC-KR')
측정항목정보 = pd.read_csv('/content/BizDataAnalysis/DATA/서울시 대기오염 측정항목 정보.csv',encoding='EUC-KR')

In [ ]:
측정소정보 = 측정소정보[['측정소 코드','측정소 이름']]

In [ ]:
측정소정보.head()

In [ ]:
측정항목정보 = 측정항목정보[['측정항목 코드','측정항목 명(줄임 명칭)']]

In [ ]:
측정항목정보.head()

* 데이터 결합

In [ ]:
seoul_air = pd.merge(seoul_air,측정소정보,how='left',on='측정소 코드')
seoul_air = pd.merge(seoul_air,측정항목정보,how='left',left_on='측정항목', right_on='측정항목 코드')

In [ ]:
seoul_air.head()

^ 서대문구의 요일별 미세먼지(PM10)의 오염도 분석

In [ ]:
sa1 = seoul_air[(seoul_air['측정소 이름'] == '서대문구') & (seoul_air['측정항목 명(줄임 명칭)'] == 'PM10')]
sa1['요일'] = sa1['측정일시'].dt.day_name()

In [ ]:
sa1.head()

In [ ]:
sa1.groupby(['요일'], as_index=False)['평균값'].mean()

* 데이터프레임의 groupby 메소드

In [ ]:
sa1.groupby(['요일'], as_index=False).size()

In [ ]:
sa1.groupby(['요일']).size()

In [ ]:
sa1.groupby(['요일']).count()

In [ ]:
sa1.groupby('요일').mean()

In [ ]:
sa1.groupby('요일', as_index=False)['평균값'].mean()

In [ ]:
sa1.groupby('요일')[['평균값']].mean()

In [ ]:
sa1.groupby('요일')[['평균값']].mean().round(2)

In [ ]:
sa1.groupby('요일')['평균값'].agg(['count','mean','median','std','min','max'])

In [ ]:
sa1.groupby('요일').agg({'국가 기준초과 구분':'count', '지자체 기준초과 구분' : 'sum', '평균값' : ['mean','min','max']})

* 특이값과 결측값 처리

In [ ]:
sa1.groupby('측정기 상태')['평균값'].agg(['min','max','mean','median']).round(3)

In [ ]:
sa1[sa1['측정기 상태'] == 0].groupby('요일')['평균값'].agg(['count','min','max','mean','median','std'])

* 서울시의 대기오염 데이터를 다양한 형태로

In [ ]:
sa2 = seoul_air[seoul_air['측정기 상태']==0]

In [ ]:
sa2 = seoul_air[['측정일시','측정소 이름','측정항목 명(줄임 명칭)','평균값']]

In [ ]:
sa2.value_counts()

In [ ]:
sa2.head()

In [ ]:
sa2.columns = ['측정일시','측정소','측정항목','측정값']

In [ ]:
sa3 = pd.pivot_table(sa2,index=['측정일시','측정소'],columns=['측정항목'],values='측정값')

In [ ]:
sa3.value_counts()

In [ ]:
sa3.head()

In [ ]:
sa4 = sa3.groupby('측정소').agg('mean')

In [ ]:
sa4.to_excel(excel_writer='서울시 대기오염.xlsx')

* 범주형 데이터를 이용한 분할표 분석

In [ ]:
sa5 = seoul_air[['측정일시','측정소 이름','측정기 상태','측정항목 명(줄임 명칭)','국가 기준초과 구분','지자체 기준초과 구분']]
sa5.columns = ['측정일시', '측정소', '측정기 상태', '측정항목', '기준초과(국가)','기준초과(지자체)']
sa5.head()

In [ ]:
pd.crosstab(index=[sa5['측정소'],sa5['측정항목']], columns=sa5['기준초과(국가)'])

In [ ]:
pd.crosstab(index=sa5['측정소'], columns=[sa5['측정항목'],sa5['기준초과(국가)']])

In [ ]:
pd.crosstab(index=sa5['측정소'], columns=[sa5['측정항목'],sa5['기준초과(국가)']],margins=True)

In [ ]:
tmp = pd.crosstab(index=sa5['측정소'], columns=[sa5['측정항목'],sa5['기준초과(국가)']], normalize='columns').round(3)

In [ ]:
tmp = tmp.drop(tmp.columns[[0,2,4,6,8,10]],axis=1)

In [ ]:
tmp